# Get a Biomass function using a GSM
**Model Publication:**  
Kugler, A., Stensjö, K. Optimal energy and redox metabolism in the cyanobacterium Synechocystis sp. PCC 6803. npj Syst Biol Appl 9, 47 (2023). https://doi.org/10.1038/s41540-023-00307-3

**Available at:**  
https://github.com/amitkugler/CBA

In [27]:
from cobra.io import load_json_model
from cobra.flux_analysis.loopless import add_loopless
from cobra.flux_analysis import pfba

from cobra import Model, Reaction, Metabolite

In [12]:
# Load the model
model = load_json_model('../CBA/GSM/JSON/iJN678_AK.json')

In [72]:
def add_import(model, metabolite_name, reaction_stoichiometry, bounds=(-1000,1000)):
    # Define the reaction
    reac = Reaction(f'{metabolite_name}_import')
    reac.name = f'{metabolite_name} import'
    reac.bounds = bounds

    # Add the correct stoichiometry and add to the model
    reac.add_metabolites({model.metabolites.get_by_id(k): v for k,v in reaction_stoichiometry.items()})
    model.add_reactions([reac])
    
    return model

In [98]:
with model:
    #############
    # Set phototrophic growth and modify the model as described in the Analysis by Kugler

    #set model objective to autotrophic growth
    model.objective = 'BIOMASS_Ec_SynAuto'
    #set photoautotropic growth by constraining glucose exchange reatcion
    model.reactions.EX_glc__D_e.bounds = (0, 1000)
    #set photoautotropic growth by constraining HCO3 exchange reatcion.
    model.reactions.EX_co2_e.bounds = (0, 1000)
    model.reactions.EX_hco3_e.bounds = (-3.7, 1000)
    #set photon flux to lab scale conditions.
    model.reactions.EX_photon_e.bounds = (-45, -45)
    #blocking transhydrogenase PntAB
    model.reactions.NADTRHD.bounds = (0, 0)

    # Make the model loopless (makes model infeasible)
    # add_loopless(model)

    # Get the default flux
    _sol = model.optimize("maximize")
    uPRE = _sol.fluxes["BIOMASS_Ec_SynAuto"]

    print(f"Autotrophic biomass flux: {uPRE}\n")

    ##############
    # Disable light reactions
    model.reactions.EX_photon_e.bounds = (0, 0)

    # Instead, add Exchange reactions for ATP, NADPH and 3PGA
    pseudo_reactions = {
        "3PGA":  ({"3pg_c":1}, (-1, 1)),
        "ATP":   ({"atp_c":1, "adp_c":-1}, (-1000, 1000)),
        "NADPH": ({"nadph_c":1, "nadp_c":-1, "h_c":-1}, (-1000, 1000))
    }
    for name, (stoich, bounds) in pseudo_reactions.items():
        model = add_import(model, name, stoich, bounds)

    # Disable other Carbon import reactions
    # HCO3 import
    # model.reactions.EX_hco3_e.lower_bound = 0

    sol = model.optimize("maximize")
    print(model.summary(sol))

    ATP_prod = sol.fluxes[[x.id for x in model.metabolites.get_by_id("atp_c").reactions]]

Autotrophic biomass flux: 0.08239889629798824

Objective
1.0 BIOMASS_Ec_SynAuto = 0.15999112222951956

Uptake
------
Metabolite     Reaction      Flux  C-Number C-Flux
     3pg_c  3PGA_import         1         3 44.77%
     ca2_e     EX_ca2_e 0.0007219         0  0.00%
 cobalt2_e EX_cobalt2_e 0.0005203         0  0.00%
     cu2_e     EX_cu2_e 0.0004813         0  0.00%
     fe2_e     EX_fe2_e  0.001198         0  0.00%
     fe3_e     EX_fe3_e   0.00109         0  0.00%
       h_e       EX_h_e     51.94         0  0.00%
    hco3_e    EX_hco3_e       3.7         1 55.22%
       k_e       EX_k_e   0.02707         0  0.00%
     mg2_e     EX_mg2_e  0.004602         0  0.00%
     mn2_e     EX_mn2_e 0.0004846         0  0.00%
    mobd_e    EX_mobd_e 0.0004847         0  0.00%
     na1_e     EX_na1_e 0.0006059         0  0.00%
     no3_e     EX_no3_e     1.442         0  0.00%
      o2_e      EX_o2_e     3.674         0  0.00%
     so4_e     EX_so4_e   0.03036         0  0.00%
     zn2_e     E

In [ ]:
sol.fluxes.get

In [106]:
sol.fluxes.get("ATP_import")

-0.8895665621019178

In [107]:
sol.fluxes.get("NADPH_import")

22.572853420093764

In [108]:
sol.fluxes.get("3PGA_import")

1.0